## Fine-tune Mistral for SUD

In [ ]:
#!pip3 install --upgrade git+https://github.com/huggingface/transformers


## Import libraries

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import numpy as np
import pandas as pd
import csv
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from peft import PeftModel
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

## Data Loading


In [8]:
# Read data from CSV file into a pandas DataFrame

filename = "/data/ARENAS_Automatic_Extremist_Analysis/ARENAS_Automatic_Extremist_Analysis/Data/SUD_data/all.csv"
df = pd.read_csv(filename)
# Rename columnns
df.rename(columns={'text': 'input', 'class': 'labels'}, inplace=True)
# Select only the "input" and "labels" columns from the DataFrame
df = df[["input", "labels"]]

In [9]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
unique_values = df['labels'].unique()

# Print the unique values
print(unique_values)

['neither' 'offensive' 'hate' 'abusive' 'profane' 'severe_toxic' 'toxic'
 'identity_hate' 'insult' 'obscene' 'threat' 'aggressive']


In [10]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
df['labels'] = df['labels'].replace({'severe_toxic': 'severe', 'identity_hate': 'identity', "insult": "ins", "offensive": "off", "profane": "prof", "obscene": "obsc", "toxic": "to"})

# Now 'severe_toxic' should be replace

## Data Splitting and Prompt Generation

In [11]:
X_train = list()
X_test = list()
# Split the data into training, testing, and evaluation sets for each sentiment label
for labels in ["off", "neither", "hate", "severe", "to", "ins", "prof", "obsc", "identity", "threat"]:
    train, test  = train_test_split(df[df.labels==labels], 
                                    train_size=552,
                                    test_size=137, 
                                    random_state=42)
    X_train.append(train)
    X_test.append(test)

# Concatenate and shuffle the training and testing sets
X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)

# Select evaluation samples from the remaining data
eval_idx = [idx for idx in df.index if idx not in list(train.index) + list(test.index)]
X_eval = df[df.index.isin(eval_idx)]
X_eval = (X_eval
          .groupby('labels', group_keys=False)
          .apply(lambda x: x.sample(n=50, random_state=10, replace=True)))

# Reset index of the training set
X_train = X_train.reset_index(drop=True)

# Function to generate prompts for training and evaluation data
def generate_prompt(data_point):
    return f"""
            Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, 
            and return the answer as the corresponding label:
            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat". 
            Make sure to give the whole label as an answer.
            [{data_point["input"]}] = {data_point["labels"]}
            """.strip()

# Function to generate prompts for testing data
def generate_test_prompt(data_point):
    return f"""
            Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, 
            and return the answer as the corresponding label:
            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".
            Make sure to give the whole label as an answer.
            [{data_point["input"]}] = """.strip()

# Convert the prompts into DataFrames for training, evaluation, and testing
X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), 
                       columns=["input"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1), 
                      columns=["input"])

y_true = X_test.labels
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["input"])

# Create datasets from the generated prompts
train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

## Evaluation function

In [12]:
def evaluate(y_true, y_pred):
    # Define labels and their corresponding numeric mappings
    labels = ["off", "neither", "hate", "severe", "to", "ins", "prof", "obsc", "identity", "threat"]
    mapping = {"off": 9, "neither": 8, "hate": 7, "severe" : 6, "to": 5, "ins": 4, "prof": 3, "obsc": 2, "identity": 1, "threat": 0}
    
    # Function to map labels to numeric values
    def map_func(x):
        return mapping.get(x, 1)
    
    # Apply mapping to true and predicted labels
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) 
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    print('\nConfusion Matrix:')
    print(conf_matrix)

## Model Configuration and Initialization

In [13]:
model_name = "mistralai/Mistral-7B-v0.1"
# Define the data type for model computation
compute_dtype = getattr(torch, "float16")

# Configure BitsAndBytes quantization parameters
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# Initialize the model with quantization settings
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config, 
)

# Initialize the model with quantization settings
model.config.use_cache = False
model.config.pretraining_tp = 1

# Initialize the tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
# Set padding token and side for the tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"




Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.17s/it]


## Prediction Function


In [14]:
def predict(test, model, tokenizer):
    """
    Make predictions using a text generation model.

    Parameters:
        test (pd.DataFrame): DataFrame containing test data.
        model: Pre-trained text generation model.
        tokenizer: Tokenizer for the model.

    Returns:
        list: Predicted labels for each input.
    """    
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["input"]
        # Create a text generation pipeline
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens = 1, 
                        temperature = 0.0,
                       )
        # Generate text based on the prompt
        result = pipe(prompt)
        #print(result)
        answer = result[0]['generated_text'].split("=")[-1]
        #print(answer)
        # Map the generated text to sentiment labels
        if "off" in answer:
            y_pred.append("off")
        elif "hate" in answer:
            y_pred.append("hate")
        elif "severe" in answer:
            y_pred.append("severe")
        elif "to" in answer:
            y_pred.append("to")
        elif "ins" in answer:
            y_pred.append("ins")
        elif "prof" in answer:
            y_pred.append("prof")
        elif "obsc" in answer:
            y_pred.append("obsc")
        elif "identity" in answer:
            y_pred.append("identity")
        elif "threat" in answer:
            y_pred.append("threat")
        else:
            y_pred.append("neither")
    return y_pred

In [15]:
y_pred = predict(test, model, tokenizer)


100%|██████████| 1370/1370 [02:56<00:00,  7.76it/s]


In [16]:
evaluate(y_true, y_pred)


Accuracy: 0.133
Accuracy for label 0: 0.000
Accuracy for label 1: 0.000
Accuracy for label 2: 0.000
Accuracy for label 3: 0.000
Accuracy for label 4: 0.000
Accuracy for label 5: 0.007
Accuracy for label 6: 0.000
Accuracy for label 7: 0.956
Accuracy for label 8: 0.365
Accuracy for label 9: 0.000

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       137
           1       0.00      0.00      0.00       137
           2       0.00      0.00      0.00       137
           3       0.00      0.00      0.00       137
           4       0.00      0.00      0.00       137
           5       0.10      0.01      0.01       137
           6       0.00      0.00      0.00       137
           7       0.13      0.96      0.22       137
           8       0.16      0.36      0.22       137
           9       0.00      0.00      0.00       137

    accuracy                           0.13      1370
   macro avg       0.04      0

## LoRA Model Configuration

In [19]:
# LoRA Model Configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"],
)
# Training Arguments Configuration
training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    evaluation_strategy="epoch"
)

# Trainer Initialization
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="input",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=1024,
)

Map: 100%|██████████| 550/550 [00:00<00:00, 2523.39 examples/s]


## Model training

In [20]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("trained-model")

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.837600,0.774154
2,0.426200,0.744191
3,0.590700,0.780067


In [21]:
y_pred = predict(test, model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 1370/1370 [06:22<00:00,  3.58it/s]

Accuracy: 0.624
Accuracy for label 0: 0.839
Accuracy for label 1: 0.723
Accuracy for label 2: 0.518
Accuracy for label 3: 0.679
Accuracy for label 4: 0.285
Accuracy for label 5: 0.511
Accuracy for label 6: 0.547
Accuracy for label 7: 0.708
Accuracy for label 8: 0.745
Accuracy for label 9: 0.686

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.84      0.82       137
           1       0.58      0.72      0.64       137
           2       0.51      0.52      0.51       137
           3       0.69      0.68      0.68       137
           4       0.39      0.28      0.33       137
           5       0.54      0.51      0.52       137
           6       0.52      0.55      0.53       137
           7       0.67      0.71      0.69       137
           8       0.78      0.74      0.76       137
           9       0.70      0.69      0.69       137

    accuracy                           0.62      1370
   macro avg       0.62      0

## Training log

In [22]:
from sklearn.metrics import classification_report, confusion_matrix

def log_training_results(model_name, bnb_config, peft_config, training_arguments, trainer_state, splitting_info, prompt_generation_info, y_true, y_pred):
    log_file = "training_log.txt"

    # Calculate classification report and confusion matrix
    classification_rep = classification_report(y_true, y_pred)
    confusion_mat = confusion_matrix(y_true, y_pred).tolist()

    # Append the current training information to the log file
    with open(log_file, 'a') as txtfile:
        txtfile.write(f"Model Name: {model_name}\n")
        txtfile.write(f"BitsAndBytes Config: {str(bnb_config)}\n")
        txtfile.write(f"Lora Config: {str(peft_config)}\n")
        txtfile.write(f"Training Arguments: {str(training_arguments)}\n")
        txtfile.write(f"Splitting Info: {splitting_info}\n")
        txtfile.write(f"Prompt Generation Info: {prompt_generation_info}\n")
        txtfile.write(f"Classification Report:\n{classification_rep}\n")
        txtfile.write(f"Confusion Matrix:\n{confusion_mat}\n\n")

    # Return the classification report
    return classification_rep


# Log training parameters, results, splitting info, prompt generation info, and prediction results
splitting_info = "Training samples: {}, Testing samples: {}, Evaluation samples: {}".format(len(X_train), len(X_test), len(X_eval))
prompt_generation_info = "Prompt generation details:  Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, and return the answer as the corresponding label: off or neither or hate or severe or to or ins or prof or obsc or identity or threat. Make sure to give the whole label as an answer."  # Add details about how prompts were generated

# Store the classification report in a text file
classification_rep = log_training_results(model_name, bnb_config, peft_config, training_arguments, trainer.state, splitting_info, prompt_generation_info, y_true, y_pred)

# Now you can access the classification report
print(classification_rep)

              precision    recall  f1-score   support

        hate       0.67      0.71      0.69       137
    identity       0.58      0.72      0.64       137
         ins       0.39      0.28      0.33       137
     neither       0.78      0.74      0.76       137
        obsc       0.51      0.52      0.51       137
         off       0.70      0.69      0.69       137
        prof       0.69      0.68      0.68       137
      severe       0.52      0.55      0.53       137
      threat       0.81      0.84      0.82       137
          to       0.54      0.51      0.52       137

    accuracy                           0.62      1370
   macro avg       0.62      0.62      0.62      1370
weighted avg       0.62      0.62      0.62      1370

